<a href="https://colab.research.google.com/github/konstantins7/rabota1/blob/main/_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-auth requests pandas google-api-python-client
import requests
import pandas as pd
import datetime
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
from googleapiclient.discovery import build

# Путь к файлам
KEY_FILE_LOCATION = '/content/medvitrina24kz-348e2b42e091.json'  # путь к загруженному JSON файлу
TXT_FILE_PATH = '/content/urlmedvitrina24.txt'  # путь к загруженному TXT файлу
PROGRESS_FILE_PATH = '/content/indexing_progress.csv'  # файл для отслеживания прогресса

SCOPES = ["https://www.googleapis.com/auth/indexing"]
INDEXING_URL = "https://indexing.googleapis.com/v3/urlNotifications:publish"
DAILY_QUOTA = 200  # Пример дневной квоты, замените на ваше значение

# Создание учетных данных и авторизованного сеанса
credentials = service_account.Credentials.from_service_account_file(KEY_FILE_LOCATION, scopes=SCOPES)
authed_session = AuthorizedSession(credentials)

# Загрузка данных из TXT файла
with open(TXT_FILE_PATH, 'r') as file:
    urls = [line.strip() for line in file]

# Создание или загрузка файла прогресса
try:
    progress = pd.read_csv(PROGRESS_FILE_PATH)
except FileNotFoundError:
    progress = pd.DataFrame(urls, columns=['Product URL'])
    progress['Indexed'] = False
    progress.to_csv(PROGRESS_FILE_PATH, index=False)

# Функция для отправки URL-адреса на индексацию
def publish_url(url):
    data = {
        "url": url,
        "type": "URL_UPDATED"
    }
    response = authed_session.post(INDEXING_URL, json=data)
    return response

# Функция для логирования сообщений
def log_message(message):
    with open("/content/indexing_log.txt", "a") as log_file:
        log_file.write(f"{datetime.datetime.now()}: {message}\n")

# Проверка квоты перед началом индексации
def check_quota():
    service = build('indexing', 'v3', credentials=credentials)
    try:
        response = service.urlNotifications().publish(body={
            "url": "https://example.com",  # Замените на ваш домен, чтобы проверить квоту
            "type": "URL_UPDATED"
        }).execute()
        return response
    except Exception as e:
        log_message(f"Ошибка при проверке квоты: {e}")
        return None

# Определение текущей даты
today = datetime.date.today()

# Отслеживание количества отправленных URL
urls_sent_today = 0

# Проверка квоты перед началом индексации
quota_info = check_quota()
if quota_info:
    log_message(f"Информация о квоте: {quota_info}")

if quota_info and 'dailyLimitRemaining' in quota_info and quota_info['dailyLimitRemaining'] > 0:
    # Обход по URL-адресам и отправка на индексацию
    for index, row in progress.iterrows():
        if urls_sent_today >= DAILY_QUOTA:
            log_message("Достигнута дневная квота. Попробуйте снова завтра.")
            print("Достигнута дневная квота. Попробуйте снова завтра.")
            break

        if not row['Indexed']:
            url = row['Product URL']
            response = publish_url(url)

            if response.status_code == 200:
                progress.at[index, 'Indexed'] = True
                urls_sent_today += 1
                log_message(f"URL отправлен успешно: {url}")
            elif response.status_code == 429:
                log_message("Достигнута дневная квота. Попробуйте снова завтра.")
                print("Достигнута дневная квота. Попробуйте снова завтра.")
                break
            elif response.status_code == 403:
                log_message(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
                print(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
                break
            else:
                log_message(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
                print(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
else:
    log_message("Достигнута дневная квота или произошла ошибка при проверке квоты.")
    print("Достигнута дневная квота или произошла ошибка при проверке квоты.")

# Сохранение обновленного файла прогресса
progress.to_csv(PROGRESS_FILE_PATH, index=False)

log_message(f"Сегодня было отправлено {urls_sent_today} URL-адресов.")
print(f"Сегодня было отправлено {urls_sent_today} URL-адресов.")


Достигнута дневная квота или произошла ошибка при проверке квоты.
Сегодня было отправлено 0 URL-адресов.
